In [77]:
import random
from random import choice

# Up to this point, the case we are interested is when there are EXACTLY 2 segments per line, so k=2

# m: the number of horizontal lines
# n: the number of vertical lines
# k: there are at most k segments per (horizontal) line

#### 1.Generating set of the stabbing inequalities IEQS1



In [78]:
def stabbing_ieqs(m, n, k):
    IEQS1 = [] # "IEQS1" stands for the set of stabbing inequalities.
    stab = (-1, )
    for j in range(m + n):
            stab += (0, )


    # Generate the first k constraints.
    # d: end points of the consecutive one's, ss: the very first constraint.
    s1 = stab[ : 1] + (1, ) + stab[2 : ]
    ss = s1[ : m+1]
    d = randrange(m+1, m+n+1)
    for i in range(m+1, d+1):
        ss += (1, )
    ss += s1[d+1 : ]
    IEQS1.append(ss)


    # Build up the other (k-1) constraints using the first one.
    a = d+1
    for i in range(k-1):
        if (a > m+n):
            IEQS1.append(s1)
        else:
            b = randrange(a, m+n+1)
            c = randrange(b, m+n+1)
            s_rest = s1[ : b]
            for j in range(b, c+1):
                s_rest += (1, )
            s_rest += s1[c+1: ]
            IEQS1.append(s_rest)
            a = c+1


    # Build up the rest constraints.
    for i in range(2, m+1):
        s = stab[ : i] + (1, ) + stab[i+1 : ]
        a = m+1
        for i in range(k):
            if (a > m+n):
                IEQS1.append(s)
            else:
                b = randrange(a, m+n+1)
                c = randrange(b, m+n+1)
                s_rest = s[ : b]
                for j in range(b, c+1):
                    s_rest += (1, )
                s_rest += s[c+1: ]
                IEQS1.append(s_rest)
                a = c+1

    # TO-DO: eventually take care of having the number of segments per line (before it's all k segments).
    return IEQS1

In [79]:
stabbing_ieqs(4, 5, 3)

[(-1, 1, 0, 0, 0, 1, 1, 0, 0, 0),
 (-1, 1, 0, 0, 0, 0, 0, 0, 1, 1),
 (-1, 1, 0, 0, 0, 0, 0, 0, 0, 0),
 (-1, 0, 1, 0, 0, 0, 0, 1, 1, 1),
 (-1, 0, 1, 0, 0, 0, 0, 0, 0, 0),
 (-1, 0, 1, 0, 0, 0, 0, 0, 0, 0),
 (-1, 0, 0, 1, 0, 1, 1, 0, 0, 0),
 (-1, 0, 0, 1, 0, 0, 0, 0, 0, 1),
 (-1, 0, 0, 1, 0, 0, 0, 0, 0, 0),
 (-1, 0, 0, 0, 1, 0, 0, 0, 0, 1),
 (-1, 0, 0, 0, 1, 0, 0, 0, 0, 0),
 (-1, 0, 0, 0, 1, 0, 0, 0, 0, 0)]

#### 2.Generating the set of bound inequalities IEQS2 & IEQS3



In [80]:
def lowerbound_ieqs(m, n): # the conditions x>=0
    IEQS2 = []
    all_zero = (0, )
    for j in range(m + n):
            all_zero += (0, )
    for i in range(1, m+n+1):
        unit_i = all_zero[ : i] + (1, ) + all_zero[i+1 : ]
        IEQS2.append( unit_i )
    return IEQS2


def upperbound_ieqs(m, n): # the conditions x<=1
    IEQS3 = []
    all_zero = (1, )
    for j in range(m + n):
        all_zero += (0, )
    for i in range(1, m+n+1):
        unit_i = all_zero[ : i] + (-1, ) + all_zero[i+1 : ]
        IEQS3.append( unit_i )
    return IEQS3

In [81]:
upperbound_ieqs(4, 5)

[(1, -1, 0, 0, 0, 0, 0, 0, 0, 0),
 (1, 0, -1, 0, 0, 0, 0, 0, 0, 0),
 (1, 0, 0, -1, 0, 0, 0, 0, 0, 0),
 (1, 0, 0, 0, -1, 0, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, -1, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, 0, -1, 0, 0, 0),
 (1, 0, 0, 0, 0, 0, 0, -1, 0, 0),
 (1, 0, 0, 0, 0, 0, 0, 0, -1, 0),
 (1, 0, 0, 0, 0, 0, 0, 0, 0, -1)]

In [90]:
def NegativeCone(m, n):
    negcone = []
    for i in range(m+n):
        vec = ()
        for j in range(m+n):
            vec += (0, )
        a = vec[ : i] + (-1, ) + vec[i+1 : ]
        negcone.append(a)
    return Cone(negcone)

In [103]:
def main(m, n, k):
    IEQS1 = stabbing_ieqs(m, n, k)
    IEQS2 = lowerbound_ieqs(m, n)
    IEQS3 = upperbound_ieqs(m, n)
    IEQS = IEQS1 + IEQS2 + IEQS3
    print('The instance is defined by %s' %IEQS)
    P = Polyhedron(ieqs = IEQS)


    # Pick up the fractional vertices!
    F = []
    for i in P.vertices():
        F.append(tuple(i))

    V_frac = []
    for j in F:
        for k in j:
            if k.is_integer() == False:
                V_frac.append(vector(j))
                break


    # Extract vertices from the output, form a list.
    V =[]
    for i in P.vertices():
        V.append(vector(i))

    # Stabbing inequalities: Ax>=1, x>=0, x<=1, transform those into the standard form \tilde{A}x<=b.
    R1 = []
    R2 = []
    R3 = []
    for i in IEQS1:
        R1.append(tuple(-1*x for x in i[1:]))
    for j in IEQS2:
        R2.append(tuple(-1*y for y in j[1:]))
    for k in IEQS3:
        R3.append(tuple(-1*z for z in k[1:]))
    # Be careful now the type here is tuple instead of vector, so we cannot do inner product.


    # Compute the "active cone" of each (fractional) vertex.
    AC = []
    for v in V_frac:
        AC_v = []
        for i in R1:
            if (v.inner_product(vector(i)) == -1):
                AC_v.append(i)
        for j in R2:
            if (v.inner_product(vector(j)) == 0):
                AC_v.append(j)
        for k in R3:
            if (v.inner_product(vector(k)) == 1):
                AC_v.append(k)
        AC.append(AC_v)

    # Now the whole thing becomes compatible! -1, 0, 1 are exactly the constants correspond to those inequalities.

    P_I = Polyhedron(vertices = list(P.integral_points()))

    # Next we try to get the inequalities representation of P_I.
    IEQS_I = []
    b = [] # constant b for each inequality
    for q in P_I.Hrepresentation():
        b.append(vector(q)[0])
        r = tuple(-1*x for x in vector(q)[1:])
        IEQS_I.append(r)
    len(IEQS_I)

    # Extract vertices from the output, form a list.
    V_I =[]
    for i in P_I.vertices():
        V_I.append(vector(i))

    # Compute the active cone of each vertices of P_I (denoted as AC_I_v).
    AC_I = []
    for v in V_I:
        AC_I_v = []
        l = len(IEQS_I)
        for i in range(l):
            if v.inner_product(vector(IEQS_I[i])) == b[i]:
                AC_I_v.append(IEQS_I[i])
        AC_I.append(AC_I_v)


    # First, we define the negative cone, to eliminate some obviously impossible vertices.
    NegCone = NegativeCone(m, n)

    # Second, list the intersections:
    IntersectionCones = []
    p = len(V_I)
    q = len(V_frac)

    for i in range(p):
        a = Cone(AC_I[i])
        for j in range(q):
            intcone1 = Cone(AC[j]).intersection(NegCone)
            intcone = a.intersection(intcone1)
            IntersectionCones.append( (V_I[i], V_frac[j], intcone.rays()) )

    R = []
    W = []
    for c in IntersectionCones:
        # Compute the greatest ratio: range(2, length)
        r = 0
        w = ()
        for i in c[2]:
            if vector(c[1]).inner_product(vector(i)) != 0:
                ratio = vector(c[0]).inner_product(vector(i)) / vector(c[1]).inner_product(vector(i))
                if ratio > r:
                    r = ratio
                    w = tuple(i)
        R.append(r)
        W.append(w)
        print('The greatest ratio of %s' %c[0], 'and %s' %c[1], 'is %f' %r, 'which is taken at weight %s' %vector(w), '\n')

In [115]:
for i in range(100):
    main(3, 5, 2)

The instance is defined by [(-1, 1, 0, 0, 1, 1, 1, 1, 1), (-1, 1, 0, 0, 0, 0, 0, 0, 0), (-1, 0, 1, 0, 0, 0, 0, 0, 1), (-1, 0, 1, 0, 0, 0, 0, 0, 0), (-1, 0, 0, 1, 0, 0, 0, 0, 1), (-1, 0, 0, 1, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1), (1, -1, 0, 0, 0, 0, 0, 0, 0), (1, 0, -1, 0, 0, 0, 0, 0, 0), (1, 0, 0, -1, 0, 0, 0, 0, 0), (1, 0, 0, 0, -1, 0, 0, 0, 0), (1, 0, 0, 0, 0, -1, 0, 0, 0), (1, 0, 0, 0, 0, 0, -1, 0, 0), (1, 0, 0, 0, 0, 0, 0, -1, 0), (1, 0, 0, 0, 0, 0, 0, 0, -1)]
The instance is defined by [(-1, 1, 0, 0, 1, 1, 1, 0, 0), (-1, 1, 0, 0, 0, 0, 0, 0, 1), (-1, 0, 1, 0, 0, 0, 1, 1, 1), (-1, 0, 1, 0, 0, 0, 0, 0, 0), (-1, 0, 0, 1, 0, 0, 0, 0, 1), (-1, 0, 0, 1, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 

The instance is defined by [(-1, 1, 0, 0, 1, 1, 0, 0, 0), (-1, 1, 0, 0, 0, 0, 1, 0, 0), (-1, 0, 1, 0, 0, 0, 0, 1, 0), (-1, 0, 1, 0, 0, 0, 0, 0, 1), (-1, 0, 0, 1, 0, 0, 0, 1, 0), (-1, 0, 0, 1, 0, 0, 0, 0, 1), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1), (1, -1, 0, 0, 0, 0, 0, 0, 0), (1, 0, -1, 0, 0, 0, 0, 0, 0), (1, 0, 0, -1, 0, 0, 0, 0, 0), (1, 0, 0, 0, -1, 0, 0, 0, 0), (1, 0, 0, 0, 0, -1, 0, 0, 0), (1, 0, 0, 0, 0, 0, -1, 0, 0), (1, 0, 0, 0, 0, 0, 0, -1, 0), (1, 0, 0, 0, 0, 0, 0, 0, -1)]


The instance is defined by [(-1, 1, 0, 0, 1, 1, 1, 1, 0), (-1, 1, 0, 0, 0, 0, 0, 0, 1), (-1, 0, 1, 0, 0, 0, 1, 1, 1), (-1, 0, 1, 0, 0, 0, 0, 0, 0), (-1, 0, 0, 1, 0, 0, 0, 1, 1), (-1, 0, 0, 1, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1), (1, -1, 0, 0, 0, 0, 0, 0, 0), (1, 0, -1, 0, 0, 0, 0, 0, 0), (1, 0, 0, -1, 0, 0, 0, 0, 0), (1, 0, 0, 0, -1, 0, 0, 0, 0), (1, 0, 0, 0, 0, -1, 0, 0, 0), (1, 0, 0, 0, 0, 0, -1, 0, 0), (1, 0, 0, 0, 0, 0, 0, -1, 0), (1, 0, 0, 0, 0, 0, 0, 0, -1)]
The instance is defined by [(-1, 1, 0, 0, 1, 1, 1, 1, 1), (-1, 1, 0, 0, 0, 0, 0, 0, 0), (-1, 0, 1, 0, 0, 1, 1, 1, 0), (-1, 0, 1, 0, 0, 0, 0, 0, 1), (-1, 0, 0, 1, 1, 1, 1, 1, 1), (-1, 0, 0, 1, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 

The instance is defined by [(-1, 1, 0, 0, 1, 1, 1, 1, 1), (-1, 1, 0, 0, 0, 0, 0, 0, 0), (-1, 0, 1, 0, 0, 0, 0, 0, 1), (-1, 0, 1, 0, 0, 0, 0, 0, 0), (-1, 0, 0, 1, 0, 1, 1, 0, 0), (-1, 0, 0, 1, 0, 0, 0, 0, 1), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1), (1, -1, 0, 0, 0, 0, 0, 0, 0), (1, 0, -1, 0, 0, 0, 0, 0, 0), (1, 0, 0, -1, 0, 0, 0, 0, 0), (1, 0, 0, 0, -1, 0, 0, 0, 0), (1, 0, 0, 0, 0, -1, 0, 0, 0), (1, 0, 0, 0, 0, 0, -1, 0, 0), (1, 0, 0, 0, 0, 0, 0, -1, 0), (1, 0, 0, 0, 0, 0, 0, 0, -1)]
The instance is defined by [(-1, 1, 0, 0, 1, 1, 1, 0, 0), (-1, 1, 0, 0, 0, 0, 0, 0, 1), (-1, 0, 1, 0, 1, 1, 0, 0, 0), (-1, 0, 1, 0, 0, 0, 1, 1, 0), (-1, 0, 0, 1, 1, 0, 0, 0, 0), (-1, 0, 0, 1, 0, 1, 1, 0, 0), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 

The greatest ratio of (0, 0, 0, 1, 0, 1, 0, 1) and (0, 0, 1/2, 1/2, 1/2, 0, 1, 1) is 1.333333 which is taken at weight (0, -1, -1, -1, -1, -1, 0, 0) 

The greatest ratio of (0, 0, 0, 1, 0, 1, 0, 1) and (1, 0, 1/2, 1/2, 1/2, 0, 1, 0) is 1.333333 which is taken at weight (0, -1, -1, -1, -1, -1, 0, 0) 

The greatest ratio of (0, 0, 0, 1, 0, 1, 0, 1) and (1, 0, 1/2, 1/2, 1/2, 0, 1, 1) is 1.333333 which is taken at weight (0, -1, -1, -1, -1, -1, 0, 0) 

The greatest ratio of (0, 0, 0, 1, 0, 1, 0, 1) and (0, 1, 1/2, 1/2, 1/2, 0, 1, 1) is 1.333333 which is taken at weight (-1, 0, -1, -1, -1, -1, 0, 0) 

The greatest ratio of (0, 0, 0, 1, 0, 1, 0, 1) and (0, 1/2, 1/2, 1/2, 0, 1/2, 1, 1) is 1.333333 which is taken at weight (-1, 0, -1, -1, -1, -1, 0, 0) 

The greatest ratio of (0, 0, 0, 1, 0, 1, 0, 1) and (0, 1/2, 1, 1/2, 0, 1/2, 0, 1) is 1.333333 which is taken at weight (-1, -1, 0, -1, -1, -1, -1, 0) 

The greatest ratio of (0, 0, 0, 1, 0, 1, 0, 1) and (0, 1/2, 1, 0, 1/2, 1/2, 0, 1) is 1.3333

...

The instance is defined by [(-1, 1, 0, 0, 1, 0, 0, 0, 0), (-1, 1, 0, 0, 0, 0, 0, 0, 1), (-1, 0, 1, 0, 0, 0, 0, 1, 0), (-1, 0, 1, 0, 0, 0, 0, 0, 1), (-1, 0, 0, 1, 0, 1, 1, 1, 0), (-1, 0, 0, 1, 0, 0, 0, 0, 1), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1), (1, -1, 0, 0, 0, 0, 0, 0, 0), (1, 0, -1, 0, 0, 0, 0, 0, 0), (1, 0, 0, -1, 0, 0, 0, 0, 0), (1, 0, 0, 0, -1, 0, 0, 0, 0), (1, 0, 0, 0, 0, -1, 0, 0, 0), (1, 0, 0, 0, 0, 0, -1, 0, 0), (1, 0, 0, 0, 0, 0, 0, -1, 0), (1, 0, 0, 0, 0, 0, 0, 0, -1)]
The instance is defined by [(-1, 1, 0, 0, 1, 1, 0, 0, 0), (-1, 1, 0, 0, 0, 0, 0, 0, 1), (-1, 0, 1, 0, 1, 1, 0, 0, 0), (-1, 0, 1, 0, 0, 0, 1, 1, 1), (-1, 0, 0, 1, 0, 1, 0, 0, 0), (-1, 0, 0, 1, 0, 0, 1, 1, 1), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 

The instance is defined by [(-1, 1, 0, 0, 1, 1, 1, 1, 1), (-1, 1, 0, 0, 0, 0, 0, 0, 0), (-1, 0, 1, 0, 0, 0, 1, 1, 1), (-1, 0, 1, 0, 0, 0, 0, 0, 0), (-1, 0, 0, 1, 0, 0, 1, 1, 0), (-1, 0, 0, 1, 0, 0, 0, 0, 1), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1), (1, -1, 0, 0, 0, 0, 0, 0, 0), (1, 0, -1, 0, 0, 0, 0, 0, 0), (1, 0, 0, -1, 0, 0, 0, 0, 0), (1, 0, 0, 0, -1, 0, 0, 0, 0), (1, 0, 0, 0, 0, -1, 0, 0, 0), (1, 0, 0, 0, 0, 0, -1, 0, 0), (1, 0, 0, 0, 0, 0, 0, -1, 0), (1, 0, 0, 0, 0, 0, 0, 0, -1)]
The instance is defined by [(-1, 1, 0, 0, 1, 1, 1, 0, 0), (-1, 1, 0, 0, 0, 0, 0, 1, 1), (-1, 0, 1, 0, 0, 0, 0, 1, 1), (-1, 0, 1, 0, 0, 0, 0, 0, 0), (-1, 0, 0, 1, 1, 1, 1, 1, 0), (-1, 0, 0, 1, 0, 0, 0, 0, 1), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 

The instance is defined by [(-1, 1, 0, 0, 1, 1, 0, 0, 0), (-1, 1, 0, 0, 0, 0, 0, 0, 1), (-1, 0, 1, 0, 1, 0, 0, 0, 0), (-1, 0, 1, 0, 0, 0, 1, 0, 0), (-1, 0, 0, 1, 0, 0, 0, 1, 0), (-1, 0, 0, 1, 0, 0, 0, 0, 1), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1), (1, -1, 0, 0, 0, 0, 0, 0, 0), (1, 0, -1, 0, 0, 0, 0, 0, 0), (1, 0, 0, -1, 0, 0, 0, 0, 0), (1, 0, 0, 0, -1, 0, 0, 0, 0), (1, 0, 0, 0, 0, -1, 0, 0, 0), (1, 0, 0, 0, 0, 0, -1, 0, 0), (1, 0, 0, 0, 0, 0, 0, -1, 0), (1, 0, 0, 0, 0, 0, 0, 0, -1)]
The instance is defined by [(-1, 1, 0, 0, 1, 1, 1, 1, 1), (-1, 1, 0, 0, 0, 0, 0, 0, 0), (-1, 0, 1, 0, 0, 1, 1, 1, 1), (-1, 0, 1, 0, 0, 0, 0, 0, 0), (-1, 0, 0, 1, 0, 0, 1, 1, 1), (-1, 0, 0, 1, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 

The instance is defined by [(-1, 1, 0, 0, 1, 1, 1, 1, 1), (-1, 1, 0, 0, 0, 0, 0, 0, 0), (-1, 0, 1, 0, 0, 0, 0, 0, 1), (-1, 0, 1, 0, 0, 0, 0, 0, 0), (-1, 0, 0, 1, 0, 0, 0, 0, 1), (-1, 0, 0, 1, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1), (1, -1, 0, 0, 0, 0, 0, 0, 0), (1, 0, -1, 0, 0, 0, 0, 0, 0), (1, 0, 0, -1, 0, 0, 0, 0, 0), (1, 0, 0, 0, -1, 0, 0, 0, 0), (1, 0, 0, 0, 0, -1, 0, 0, 0), (1, 0, 0, 0, 0, 0, -1, 0, 0), (1, 0, 0, 0, 0, 0, 0, -1, 0), (1, 0, 0, 0, 0, 0, 0, 0, -1)]
The instance is defined by [(-1, 1, 0, 0, 1, 1, 0, 0, 0), (-1, 1, 0, 0, 0, 0, 1, 0, 0), (-1, 0, 1, 0, 0, 0, 0, 0, 1), (-1, 0, 1, 0, 0, 0, 0, 0, 0), (-1, 0, 0, 1, 0, 0, 0, 1, 0), (-1, 0, 0, 1, 0, 0, 0, 0, 1), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 

The greatest ratio of (0, 1, 0, 1, 0, 1, 0, 1) and (0, 1, 1/2, 1, 0, 1/2, 0, 1/2) is 1.333333 which is taken at weight (-1, 0, -1, 0, -1, -1, -1, -1) 

The greatest ratio of (0, 1, 0, 1, 0, 1, 1, 1) and (0, 1, 1/2, 1, 0, 1/2, 0, 1/2) is 1.000000 which is taken at weight (0, -1, 0, 0, 0, 0, 0, 0) 

The greatest ratio of (0, 1, 0, 1, 1, 0, 0, 1) and (0, 1, 1/2, 1, 0, 1/2, 0, 1/2) is 1.333333 which is taken at weight (-1, 0, -1, 0, -1, -1, -1, -1) 

The greatest ratio of (0, 1, 0, 1, 1, 0, 1, 1) and (0, 1, 1/2, 1, 0, 1/2, 0, 1/2) is 1.000000 which is taken at weight (0, -1, 0, 0, 0, 0, 0, 0) 

The greatest ratio of (0, 1, 0, 1, 1, 1, 0, 1) and (0, 1, 1/2, 1, 0, 1/2, 0, 1/2) is 1.000000 which is taken at weight (0, -1, 0, 0, 0, 0, 0, 0) 

The greatest ratio of (0, 1, 0, 1, 1, 1, 1, 1) and (0, 1, 1/2, 1, 0, 1/2, 0, 1/2) is 1.000000 which is taken at weight (0, -1, 0, 0, 0, 0, 0, 0) 

The greatest ratio of (0, 1, 1, 1, 0, 0, 0, 1) and (0, 1, 1/2, 1, 0, 1/2, 0, 1/2) is 1.333333 which is taken

In [0]:
# The problem with this program is: too many easy case, which has no fractional vertices! (I guess)